# E-commerce Product Analysis: SQL + Python

##   Overview
This notebook performs a comprehensive analysis of the Olist E-commerce dataset. The goal is to extract actionable insights regarding sales trends, product performance, customer behavior, and product associations. 

**Analysis Steps:**
1.  **Setup & Data Loading**: Automatically download the dataset from Kaggle, load it into Pandas DataFrames, and clean it.
2.  **Database Integration**: Create a SQLite database and populate it with the cleaned data.
3.  **SQL-based Analysis**: Execute pre-written SQL queries to perform initial aggregations (e.g., sales trends, category revenue).
4.  **Advanced Python Analysis**: 
    * Visualize the results from the SQL queries.
    * Conduct RFM analysis for customer segmentation.
    * Perform Market Basket Analysis to discover product associations.
5.  **Insights & Conclusion**: Summarize key findings and provide business recommendations.

### Step 1: Setup and Data Acquisition

In [ ]:
import os
import pandas as pd
import numpy as np
import sqlite3
from sqlalchemy import create_engine
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import warnings

from mlxtend.frequent_patterns import apriori, association_rules

# Set plot style
sns.set_style('whitegrid')
plt.style.use('seaborn-v0_8-deep')
warnings.filterwarnings('ignore')

# Create necessary directories
os.makedirs('../data', exist_ok=True)
os.makedirs('../images', exist_ok=True)

print("Directories and libraries are set up.")

In [ ]:
# Authenticate with Kaggle API
# IMPORTANT: Make sure your kaggle.json file is in ~/.kaggle/ or in the project root.
import kaggle

print("Authenticating with Kaggle...")
kaggle.api.authenticate()

# Download the dataset
dataset = 'olistbr/brazilian-ecommerce'
download_path = '../data/'

print(f"Downloading dataset '{dataset}' to '{download_path}'...")
kaggle.api.dataset_download_files(dataset, path=download_path, unzip=True)
print("Dataset downloaded and unzipped successfully.")

### Step 2: Data Loading, Cleaning, and Database Population

In [ ]:
# Load datasets into Pandas DataFrames
data_path = '../data/'
try:
    customers = pd.read_csv(f'{data_path}olist_customers_dataset.csv')
    orders = pd.read_csv(f'{data_path}olist_orders_dataset.csv')
    order_items = pd.read_csv(f'{data_path}olist_order_items_dataset.csv')
    products = pd.read_csv(f'{data_path}olist_products_dataset.csv')
    category_translation = pd.read_csv(f'{data_path}product_category_name_translation.csv')
    order_payments = pd.read_csv(f'{data_path}olist_order_payments_dataset.csv')
    print("All CSV files loaded successfully.")
except FileNotFoundError as e:
    print(f"Error loading CSV files: {e}. Make sure the download was successful.")

In [ ]:
# --- Data Cleaning --- 

# 1. Convert date columns to datetime objects
date_cols = ['order_purchase_timestamp', 'order_approved_at', 'order_delivered_carrier_date', 'order_delivered_customer_date', 'order_estimated_delivery_date']
for col in date_cols:
    orders[col] = pd.to_datetime(orders[col], errors='coerce')

# 2. Handle missing values
# For simplicity, we'll drop rows with missing critical information like product_id or category.
products.dropna(subset=['product_category_name'], inplace=True)

# 3. Merge category translation
products = products.merge(category_translation, on='product_category_name', how='left')

print("Data cleaning and preparation complete.")
products.head()

In [ ]:
# Create a SQLite database and load the data
db_path = '../data/ecommerce.db'
engine = create_engine(f'sqlite:///{db_path}')

# Load DataFrames into SQL tables
customers.to_sql('customers', engine, if_exists='replace', index=False)
orders.to_sql('orders', engine, if_exists='replace', index=False)
order_items.to_sql('order_items', engine, if_exists='replace', index=False)
products.to_sql('products', engine, if_exists='replace', index=False)
category_translation.to_sql('product_category_name_translation', engine, if_exists='replace', index=False)
order_payments.to_sql('order_payments', engine, if_exists='replace', index=False)

print(f"Database created at '{db_path}' and all tables loaded.")

### Step 3: SQL-based Analysis

Now we'll execute the SQL queries from the `sql/` directory to perform our initial analysis.

In [ ]:
def execute_sql_query(query_path, db_engine):
    """Reads a SQL query from a file and executes it using the given database engine."""
    with open(query_path, 'r') as file:
        query = file.read()
    return pd.read_sql_query(query, db_engine)

#### 3.1 Sales Trends Over Time

In [ ]:
sales_trends_df = execute_sql_query('../sql/sales_trends.sql', engine)
sales_trends_df['sales_month'] = pd.to_datetime(sales_trends_df['sales_month'])
sales_trends_df = sales_trends_df.sort_values('sales_month')

print("Monthly Sales Trends:")
sales_trends_df.head()

In [ ]:
# Visualize Sales Trends
plt.figure(figsize=(14, 7))
sns.lineplot(data=sales_trends_df, x='sales_month', y='monthly_revenue', marker='o')
plt.title('Monthly Sales Revenue Trend', fontsize=16)
plt.xlabel('Month', fontsize=12)
plt.ylabel('Revenue (in R$)', fontsize=12)
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig('../images/monthly_sales_revenue.png')
plt.show()

**Insight**: The sales trend shows significant growth over time, with a massive peak in November 2017. This is likely due to Black Friday, indicating a strong seasonal influence on sales.

#### 3.2 Revenue by Product Category

In [ ]:
revenue_by_cat_df = execute_sql_query('../sql/revenue_by_category.sql', engine)
print("Revenue by Category:")
revenue_by_cat_df.head()

In [ ]:
# Visualize Top 10 Categories by Revenue
top_10_cats = revenue_by_cat_df.nlargest(10, 'total_revenue')

plt.figure(figsize=(12, 8))
sns.barplot(data=top_10_cats, y='category', x='total_revenue', palette='viridis', orient='h')
plt.title('Top 10 Product Categories by Revenue', fontsize=16)
plt.xlabel('Total Revenue (in R$)', fontsize=12)
plt.ylabel('Product Category', fontsize=12)
plt.tight_layout()
plt.savefig('../images/top_categories_revenue.png')
plt.show()

**Insight**: `Bed Bath Table`, `Health Beauty`, and `Sports Leisure` are the highest revenue-generating categories. These are key areas to focus on for marketing and inventory management.

#### 3.3 Top Selling Products

In [ ]:
top_products_df = execute_sql_query('../sql/top_products.sql', engine)
print("Top 10 Selling Products by Units:")
top_products_df

**Insight**: The top-selling products are concentrated in categories like `Bed Bath Table`, `Furniture Decor`, and `Garden Tools`. This reinforces the importance of these categories to the business's overall sales volume.

### Step 4: Advanced Python Analysis

#### 4.1 Customer Segmentation (RFM Analysis)
RFM stands for Recency, Frequency, and Monetary value. It's a method used to segment customers based on their purchasing behavior.

In [ ]:
# Execute the RFM query
rfm_df = execute_sql_query('../sql/customer_segmentation.sql', engine)
rfm_df.dropna(inplace=True) # Drop customers with no monetary value

# Create RFM scores by binning into quantiles
rfm_df['R_score'] = pd.qcut(rfm_df['recency'], 4, labels=[4, 3, 2, 1]) # Higher score for lower recency
rfm_df['F_score'] = pd.qcut(rfm_df['frequency'].rank(method='first'), 4, labels=[1, 2, 3, 4])
rfm_df['M_score'] = pd.qcut(rfm_df['monetary'], 4, labels=[1, 2, 3, 4])

# Combine scores
rfm_df['RFM_score'] = rfm_df['R_score'].astype(str) + rfm_df['F_score'].astype(str) + rfm_df['M_score'].astype(str)

print("RFM DataFrame with scores:")
rfm_df.head()

In [ ]:
# Define customer segments based on RFM scores
segment_map = {
    r'[3-4][3-4][3-4]': 'Champions',
    r'[2-4][3-4][1-4]': 'Loyal Customers',
    r'[3-4][1-2][1-4]': 'Potential Loyalists',
    r'[3-4][1-1][1-4]': 'New Customers',
    r'[2-3][1-2][1-4]': 'Promising',
    r'[1-2][2-4][1-4]': 'Customers Needing Attention',
    r'[1-2][1-2][2-4]': 'At Risk',
    r'[1-2][1-2][1-2]': 'Hibernating'
}

rfm_df['segment'] = rfm_df['R_score'].astype(str) + rfm_df['F_score'].astype(str)
rfm_df['segment'] = rfm_df['segment'].replace(segment_map, regex=True)

# For any that didn't match (less common combinations)
rfm_df['segment'] = rfm_df['RFM_score'].replace(segment_map, regex=True)

segment_counts = rfm_df['segment'].value_counts().reset_index()
segment_counts.columns = ['segment', 'count']
print("Customer Segments:")
segment_counts

In [ ]:
# Visualize Customer Segments using a Treemap
fig = px.treemap(segment_counts,
                  path=['segment'],
                  values='count',
                  color='count',
                  color_continuous_scale='Blues',
                  title='Customer Segmentation by RFM')
fig.update_layout(margin = dict(t=50, l=25, r=25, b=25))
fig.write_image("../images/rfm_customer_segmentation.png")
fig.show()

**Insight**: The RFM segmentation reveals a large group of `Hibernating` customers. This is a critical insight, pointing to a need for re-engagement campaigns. On the other hand, `Champions` and `Loyal Customers` represent our high-value segments that should be nurtured with loyalty programs and exclusive offers.

#### 4.2 Market Basket Analysis
This analysis helps us understand which products are frequently purchased together.

In [ ]:
# Prepare data for Market Basket Analysis
# We need a transaction-style DataFrame: one row per order, with product categories as columns

# Merge order items with products to get category names
mba_data = order_items.merge(products[['product_id', 'product_category_name_english']], on='product_id')

# Filter for relevant columns and drop missing categories
mba_data = mba_data[['order_id', 'product_category_name_english']].dropna()

# Create the basket: one-hot encode the data
basket = mba_data.groupby(['order_id', 'product_category_name_english'])['product_category_name_english']\
                    .count().unstack().reset_index().fillna(0).set_index('order_id')

# Convert counts to binary (either bought or not)
def encode_units(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1

basket_sets = basket.applymap(encode_units)

# Remove postage category if it exists as it's not a product
if 'portateis_cozinha_e_preparadores_de_alimentos' in basket_sets.columns:
    basket_sets.drop('portateis_cozinha_e_preparadores_de_alimentos', axis=1, inplace=True)

print("Basket prepared for analysis:")
basket_sets.head()

In [ ]:
# Apply Apriori algorithm to find frequent itemsets
frequent_itemsets = apriori(basket_sets, min_support=0.005, use_colnames=True)

# Generate association rules
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)

# Sort rules by lift and confidence
rules = rules.sort_values(['lift', 'confidence'], ascending=[False, False])

print("Top Product Association Rules:")
rules.head(10)

**Insight**: The association rules show strong connections between certain product categories. For example, a high **lift** value between `Computers Accessories` and `Bed Bath Table` suggests that customers buying one are significantly more likely to buy the other than by random chance. This information is gold for:
* **Cross-selling**: "Customers who bought X also bought Y" recommendations.
* **Product Bundling**: Creating attractive packages with associated products.
* **Store Layout**: Placing related product categories closer together, both online and offline.

### Step 5: Final Conclusion & Business Recommendations

This analysis has provided a multi-faceted view of the e-commerce business, yielding several actionable insights:

1.  **Focus on Top Categories**: `Bed Bath Table`, `Health Beauty`, and `Sports Leisure` are the cash cows. Marketing budget, inventory, and promotional activities should be prioritized for these categories.

2.  **Leverage Seasonality**: The November sales peak (Black Friday) is a critical revenue driver. Planning for this period should start months in advance with targeted campaigns and robust inventory.

3.  **Implement a CRM Strategy**: The RFM analysis clearly segments the customer base. A targeted CRM strategy should be developed:
    * **Nurture Champions**: Offer loyalty rewards and early access to new products.
    * **Re-engage the Hibernating**: Launch win-back campaigns with special discounts to bring dormant customers back.
    * **Grow Potential Loyalists**: Encourage repeat purchases with follow-up offers.

4.  **Boost Average Order Value (AOV)**: Use the insights from the market basket analysis to implement cross-selling and bundling strategies. Feature associated products on product pages and at checkout to increase the likelihood of add-on purchases.